# Final Project

## Initial Thoughts:
- Merge football teams with duplicate names
    - Teams that move locations
    - Teams that do no have the same team name (the ID of the favorite team may not always refer to the same team)
- Look at data to see which columns have enough data to conduct data analysis on
    - Some games have no weather data
    - Some stadiums have a lot of null information. Others are closed
- Need to decide what year to cutoff the data (relevance)

In [4]:
import pandas as pd
import numpy as np

In [5]:
spreadspoke_df = pd.read_csv('NFL_sportsbetting_data/spreadspoke_scores.csv')
teams_df = pd.read_csv('NFL_sportsbetting_data/nfl_teams.csv')
stadiums_df = pd.read_csv('NFL_sportsbetting_data/nfl_stadiums.csv', encoding='ISO-8859-1')

In [6]:
spreadspoke_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14073 entries, 0 to 14072
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   schedule_date        14073 non-null  object 
 1   schedule_season      14073 non-null  int64  
 2   schedule_week        14073 non-null  object 
 3   schedule_playoff     14073 non-null  bool   
 4   team_home            14073 non-null  object 
 5   score_home           13801 non-null  float64
 6   score_away           13801 non-null  float64
 7   team_away            14073 non-null  object 
 8   team_favorite_id     11322 non-null  object 
 9   spread_favorite      11322 non-null  float64
 10  over_under_line      11312 non-null  object 
 11  stadium              14073 non-null  object 
 12  stadium_neutral      14073 non-null  bool   
 13  weather_temperature  12525 non-null  float64
 14  weather_wind_mph     12509 non-null  float64
 15  weather_humidity     8476 non-null  

In [7]:
teams_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   team_name                44 non-null     object
 1   team_name_short          44 non-null     object
 2   team_id                  44 non-null     object
 3   team_id_pfr              44 non-null     object
 4   team_conference          44 non-null     object
 5   team_division            35 non-null     object
 6   team_conference_pre2002  44 non-null     object
 7   team_division_pre2002    42 non-null     object
dtypes: object(8)
memory usage: 2.9+ KB


In [8]:
stadiums_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   stadium_name                     120 non-null    object 
 1   stadium_location                 119 non-null    object 
 2   stadium_open                     90 non-null     float64
 3   stadium_close                    41 non-null     float64
 4   stadium_type                     109 non-null    object 
 5   stadium_address                  102 non-null    object 
 6   stadium_weather_station_zipcode  101 non-null    object 
 7   stadium_weather_type             117 non-null    object 
 8   stadium_capacity                 54 non-null     object 
 9   stadium_surface                  68 non-null     object 
 10  stadium_weather_station          62 non-null     object 
 11  stadium_weather_station_name     63 non-null     object 
 12  stadium_latitude      

In [9]:
# display(spreadspoke_df)
# display(teams_df)
# display(stadiums_df)

In [10]:
#need to make sure that data has non null values for the betting data
spreadspoke_df = spreadspoke_df.dropna(subset=['team_favorite_id','spread_favorite','over_under_line'])

#converting the date column to datetimes
spreadspoke_df['schedule_date'] = pd.to_datetime(spreadspoke_df['schedule_date'], format='%m/%d/%Y')

#dropping games before 2002
spreadspoke_df = spreadspoke_df[spreadspoke_df['schedule_season'] >= 2002]

spreadspoke_df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
7872,2002-09-05,2002,1,False,New York Giants,13.0,16.0,San Francisco 49ers,SF,-4.0,39,Giants Stadium,False,75.0,12.0,58.0,NaN
7873,2002-09-08,2002,1,False,Buffalo Bills,31.0,37.0,New York Jets,NYJ,-3.0,43,Ralph Wilson Stadium,False,75.0,7.0,50.0,NaN
7874,2002-09-08,2002,1,False,Carolina Panthers,10.0,7.0,Baltimore Ravens,PICK,0.0,33,Bank of America Stadium,False,73.0,8.0,66.0,NaN
7875,2002-09-08,2002,1,False,Chicago Bears,27.0,23.0,Minnesota Vikings,CHI,-4.5,41,Memorial Stadium (Champaign),False,76.0,5.0,75.0,NaN
7876,2002-09-08,2002,1,False,Cincinnati Bengals,6.0,34.0,San Diego Chargers,CIN,-3.0,37,Paul Brown Stadium,False,81.0,5.0,50.0,NaN


In [11]:
# Values to replace
values_to_replace = ['Washington Redskins', 'Washington Football Team']

# Replace specific values with 'New Team Name'
spreadspoke_df['team_home'] = spreadspoke_df['team_home'].replace(values_to_replace, 'Washington Commanders')  
spreadspoke_df['team_away'] = spreadspoke_df['team_away'].replace(values_to_replace, 'Washington Commanders')  

In [12]:
nfl_teams = [
    "Arizona Cardinals",  "Atlanta Falcons",  "Baltimore Ravens",  "Buffalo Bills",  "Carolina Panthers",  "Chicago Bears",
    "Cincinnati Bengals", "Cleveland Browns", "Dallas Cowboys", "Denver Broncos", "Detroit Lions", "Green Bay Packers",
    "Houston Texans", "Indianapolis Colts",  "Jacksonville Jaguars",  "Kansas City Chiefs",  "Las Vegas Raiders",
    "Los Angeles Chargers",  "Los Angeles Rams",  "Miami Dolphins",  "Minnesota Vikings",  "New England Patriots",
    "New Orleans Saints",  "New York Giants",  "New York Jets",  "Philadelphia Eagles",  "Pittsburgh Steelers",
    "San Francisco 49ers",  "Seattle Seahawks",  "Tampa Bay Buccaneers",  "Tennessee Titans",  "Washington Commanders"
]

# Create regex pattern to match any of the NFL team names
pattern = r'\b(' + '|'.join(nfl_teams) + r')\b'

# Filter the DataFrame to only keep rows that contain an NFL team name
filtered_df = spreadspoke_df[spreadspoke_df['team_home'].str.contains(pattern, case=False, regex=True)]
spreadspoke_df2 = filtered_df[filtered_df['team_away'].str.contains(pattern, case=False, regex=True)]

# Display the filtered DataFrame
spreadspoke_df2.head()

#we don't lose washington commanders as the home team here
len(list(spreadspoke_df2['team_home'].unique())) 

C:\Users\2024.WK5CG0217Y8KMID\AppData\Local\Temp\ipykernel_5952\1728587709.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = spreadspoke_df[spreadspoke_df['team_home'].str.contains(pattern, case=False, regex=True)]
C:\Users\2024.WK5CG0217Y8KMID\AppData\Local\Temp\ipykernel_5952\1728587709.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  spreadspoke_df2 = filtered_df[filtered_df['team_away'].str.contains(pattern, case=False, regex=True)]


32

In [13]:
#replace washington redskins and washington football team with washington commanders
spreadspoke_df2.loc[spreadspoke_df2['team_home'].isin(values_to_replace), 'team_home'] = 'Washington Commanders'
spreadspoke_df2.loc[spreadspoke_df2['team_away'].isin(values_to_replace), 'team_away'] = 'Washington Commanders'

#prove that it was replaced correctly
spreadspoke_df2.query("team_away == 'Washington Commanders'")
spreadspoke_df2.query("team_home == 'Washington Commanders'")

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
7886,2002-09-08,2002,1,False,Washington Commanders,31.0,23.0,Arizona Cardinals,WAS,-6.0,44.5,FedEx Field,False,73.0,5.0,70.0,NaN
7903,2002-09-16,2002,2,False,Washington Commanders,7.0,37.0,Philadelphia Eagles,PHI,-3.0,44,FedEx Field,False,75.0,12.0,84.0,NaN
7958,2002-10-13,2002,6,False,Washington Commanders,27.0,43.0,New Orleans Saints,NO,-1.5,45.5,FedEx Field,False,65.0,7.0,92.0,NaN
7986,2002-10-27,2002,8,False,Washington Commanders,26.0,21.0,Indianapolis Colts,WAS,-1.0,46,FedEx Field,False,58.0,8.0,74.0,NaN
8078,2002-12-08,2002,14,False,Washington Commanders,21.0,27.0,New York Giants,WAS,-3.0,37.5,FedEx Field,False,36.0,6.0,70.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13636,2023-10-29,2023,8,False,Washington Commanders,31.0,38.0,Philadelphia Eagles,PHI,-7.0,43,FedEx Field,False,NaN,NaN,NaN,NaN
13678,2023-11-19,2023,11,False,Washington Commanders,19.0,31.0,New York Giants,WAS,-8.5,39,FedEx Field,False,NaN,NaN,NaN,NaN
13707,2023-12-03,2023,13,False,Washington Commanders,15.0,45.0,Miami Dolphins,MIA,-9.0,49.5,FedEx Field,False,NaN,NaN,NaN,NaN
13771,2023-12-31,2023,17,False,Washington Commanders,10.0,27.0,San Francisco 49ers,SF,-14.0,48.5,FedEx Field,False,NaN,NaN,NaN,NaN


In [14]:
#Matching teams to stadiums:

#list of current stadiums that are open (including stadiums that have had multiple names since 2002)
stadiums = [
    "Acrisure Stadium", "Allegiant Stadium", "GEHA Field at Arrowhead Stadium", "AT&T Stadium", "Bank of America Stadium",
    "Caesars Superdome", "Empower Field at Mile High", "FedEx Field","FirstEnergy Stadium", "Ford Field", "Gillette Stadium",
    "Hard Rock Stadium", "Highmark Stadium", "Lambeau Field", "Levi's Stadium", "Lincoln Financial Field","Lumen Field",
    "M&T Bank Stadium", "Mercedes-Benz Stadium", "MetLife Stadium", "NRG Stadium", "Nissan Stadium", "Paycor Stadium",
    "Raymond James Stadium", "SoFi Stadium", "Soldier Field", "State Farm Stadium", "TIAA Bank Field", "U.S. Bank Stadium",
    "Lucas Oil Stadium",
    "Cowboys Stadium","Sports Authority Field at Mile High","Mile High Stadium","University of Phoenix Stadium",
    "Reliant Stadium", "EverBank Field", "Arrowhead Stadium", "LP Stadium", "Louisiana Superdome",
    "Mercedes-Benz Superdome","Bills Stadium", "New Era Field","Ralph Wilson Stadium", "Heinz Field"]

pattern = r'\b(' + '|'.join(stadiums) + r')\b'
stadiums_filtered_df = stadiums_df[stadiums_df['stadium_name'].str.contains(pattern, case=False, regex=True)]
stadiums_filtered_df = stadiums_filtered_df.drop(columns=['stadium_close', 'stadium_address', 'stadium_weather_station_zipcode', 'stadium_weather_station',	'stadium_weather_station_name', 'stadium_azimuthangle', 'stadium_elevation']).reset_index().drop(columns=['index'])
stadiums_filtered_df

C:\Users\2024.WK5CG0217Y8KMID\AppData\Local\Temp\ipykernel_5952\763062031.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  stadiums_filtered_df = stadiums_df[stadiums_df['stadium_name'].str.contains(pattern, case=False, regex=True)]


,stadium_name,stadium_location,stadium_open,stadium_type,stadium_weather_type,stadium_capacity,stadium_surface,stadium_latitude,stadium_longitude
0,AT&T Stadium,"Arlington, TX",2009.0,retractable,indoor,"80,000",FieldTurf,32.747778,-97.092778
1,Cowboys Stadium,"Arlington, TX",2009.0,retractable,indoor,"80,000",FieldTurf,32.757200,-97.073600
2,Mercedes-Benz Stadium,"Atlanta, GA",2017.0,retractable,indoor,71000,FieldTurf,33.755556,-84.400000
3,M&T Bank Stadium,"Baltimore, MD",1998.0,outdoor,moderate,"71,008",FieldTurf,39.278056,-76.622778
4,Bank of America Stadium,"Charlotte, NC",1996.0,outdoor,moderate,"73,778",Grass,35.225833,-80.852778
5,Soldier Field,"Chicago, IL",1926.0,outdoor,cold,"61,500",Grass,41.862300,-87.616700
6,Paycor Stadium,"Cincinnati, OH",2000.0,outdoor,cold,"65,515",FieldTurf,39.100000,-84.516670
7,FirstEnergy Stadium,"Cleveland, OH",1999.0,outdoor,cold,"68,000",Grass,41.506111,-81.699444
8,Empower Field at Mile High,"Denver, CO",2001.0,outdoor,cold,"76,125",Grass,39.743889,-105.020000
9,Mile High Stadium,"Denver, CO",1960.0,outdoor,cold,NaN,NaN,39.746111,-105.021667


In [15]:
#add a column to the spreadspoke_df to match the hometown to the home team
stadium_locations = {'Pittsburgh Steelers':'Pittsburgh, PA', 'Las Vegas Raiders':'Paradise, NV', 'Dallas Cowboys':'Arlington, TX',
                     'Carolina Panthers':'Charlotte, NC', 'New Orleans Saints':'New Orleans, LA', 'Denver Broncos':'Denver, CO',
                     'Washington Commanders':'Landover, MD', 'Cleveland Browns':'Cleveland, OH', 'Detroit Lions':'Detroit, MI',
                     'Kansas City Chiefs':'Kansas City, MO', 'New England Patriots':'Foxborough, MA', 
                     'Miami Dolphins':'Miami Gardens, FL', 'Buffalo Bills':'Orchard Park, NY', 'Green Bay Packers':'Green Bay, WI',
                     'San Francisco 49ers':'Santa Clara, CA', 'Philadelphia Eagles':'Philadelphia, PA', 
                     'Indianapolis Colts':'Indianapolis, IN', 'Seattle Seahawks':'Seattle, WA', 'Baltimore Ravens':'Baltimore, MD',
                     'Atlanta Falcons':'Atlanta, GA', 'New York Giants':'East Rutherford, NJ', 'New York Jets':'East Rutherford, NJ',
                     'Tennessee Titans':'Nashville, TN', 'Houston Texans':'Houston, TX', 'Cincinnati Bengals':'Cincinnati, OH',
                     'Tampa Bay Buccaneers':'Tampa, FL', 'Los Angeles Chargers':'Inglewood, CA', 'Los Angeles Rams':'Inglewood, CA',
                     'Chicago Bears':'Chicago, IL', 'Arizona Cardinals':'Glendale, AZ', 'Jacksonville Jaguars':'Jacksonville, FL',
                     'Minnesota Vikings':'Minneapolis, MN'}

spreadspoke_df2['stadium_location'] = spreadspoke_df2['team_home'].map(stadium_locations) #this is where you lose the commanders
spreadspoke_df2.reset_index()
spreadspoke_df2.head()

C:\Users\2024.WK5CG0217Y8KMID\AppData\Local\Temp\ipykernel_5952\3396990784.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spreadspoke_df2['stadium_location'] = spreadspoke_df2['team_home'].map(stadium_locations) #this is where you lose the commanders


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,stadium_location
7872,2002-09-05,2002,1,False,New York Giants,13.0,16.0,San Francisco 49ers,SF,-4.0,39,Giants Stadium,False,75.0,12.0,58.0,NaN,"East Rutherford, NJ"
7873,2002-09-08,2002,1,False,Buffalo Bills,31.0,37.0,New York Jets,NYJ,-3.0,43,Ralph Wilson Stadium,False,75.0,7.0,50.0,NaN,"Orchard Park, NY"
7874,2002-09-08,2002,1,False,Carolina Panthers,10.0,7.0,Baltimore Ravens,PICK,0.0,33,Bank of America Stadium,False,73.0,8.0,66.0,NaN,"Charlotte, NC"
7875,2002-09-08,2002,1,False,Chicago Bears,27.0,23.0,Minnesota Vikings,CHI,-4.5,41,Memorial Stadium (Champaign),False,76.0,5.0,75.0,NaN,"Chicago, IL"
7877,2002-09-08,2002,1,False,Cleveland Browns,39.0,40.0,Kansas City Chiefs,CLE,-2.0,36,FirstEnergy Stadium,False,78.0,7.0,54.0,NaN,"Cleveland, OH"


In [16]:
#merge the stadium dataframe to the spreadspoke dataframe to get the stadium information in the same df as the game information
spreadspoke_df3 = pd.merge(spreadspoke_df2, stadiums_filtered_df, on='stadium_location', how='outer')

#create a mask to filter rows for games played in stadiums that no longer exist
mask = spreadspoke_df3['schedule_season'] >= spreadspoke_df3['stadium_open']
spreadspoke_df3 = spreadspoke_df3[mask]

#need to see all of the columns
pd.set_option('display.max_columns', None)

#check for the Los Angeles Chargers
#check for the Los Angeles Rams
#check for the Las Vegas Raiders
spreadspoke_df3.query("team_home == 'Los Angeles Chargers'").head(5)
spreadspoke_df3.query("team_home == 'Los Angeles Rams'").head(5)
#spreadspoke_df3.query("team_home == 'Las Vegas Raiders'").head(5)

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,stadium_location,stadium_name,stadium_open,stadium_type,stadium_weather_type,stadium_capacity,stadium_surface,stadium_latitude,stadium_longitude
7481,2020-09-13,2020,1,False,Los Angeles Rams,20.0,17.0,Dallas Cowboys,DAL,-2.0,51.5,SoFi Stadium,False,72.0,0.0,NaN,indoor,"Inglewood, CA",SoFi Stadium,2020.0,outdoor,warm,70240,Hellas Matrix Turf,33.95345,-118.3392
7484,2020-10-04,2020,4,False,Los Angeles Rams,17.0,9.0,New York Giants,LAR,-13.0,48.5,SoFi Stadium,False,72.0,0.0,NaN,indoor,"Inglewood, CA",SoFi Stadium,2020.0,outdoor,warm,70240,Hellas Matrix Turf,33.95345,-118.3392
7486,2020-10-26,2020,7,False,Los Angeles Rams,24.0,10.0,Chicago Bears,LAR,-6.0,45,SoFi Stadium,False,72.0,0.0,NaN,indoor,"Inglewood, CA",SoFi Stadium,2020.0,outdoor,warm,70240,Hellas Matrix Turf,33.95345,-118.3392
7488,2020-11-15,2020,10,False,Los Angeles Rams,23.0,16.0,Seattle Seahawks,LAR,-3.0,54.5,SoFi Stadium,False,72.0,0.0,NaN,indoor,"Inglewood, CA",SoFi Stadium,2020.0,outdoor,warm,70240,Hellas Matrix Turf,33.95345,-118.3392
7490,2020-11-29,2020,12,False,Los Angeles Rams,20.0,23.0,San Francisco 49ers,LAR,-6.5,44.5,SoFi Stadium,False,72.0,0.0,NaN,indoor,"Inglewood, CA",SoFi Stadium,2020.0,outdoor,warm,70240,Hellas Matrix Turf,33.95345,-118.3392


In [38]:
#check to make sure we didn't lose any stadiums during the merge
unique_stadiums = spreadspoke_df3['stadium_name'].unique()

#the number of stadiums stays the same (we didn't lose any)
print(len(stadiums)) #44
print(len(unique_stadiums)) #44

44
44


In [42]:
final_df = spreadspoke_df3.reset_index().drop(columns=['index'])
final_df
#final_df contains a combination of the stadium data and the game data

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,stadium_location,stadium_name,stadium_open,stadium_type,stadium_weather_type,stadium_capacity,stadium_surface,stadium_latitude,stadium_longitude
0,2010-09-12,2010,1,False,New York Giants,31.0,18.0,Carolina Panthers,NYG,-6.0,41,MetLife Stadium,False,65.0,1.0,67.0,rain,"East Rutherford, NJ",MetLife Stadium,2010.0,outdoor,cold,"82,500",FieldTurf,40.813528,-74.074361
1,2010-09-13,2010,1,False,New York Jets,9.0,10.0,Baltimore Ravens,NYJ,-1.0,36.5,MetLife Stadium,False,73.0,1.0,100.0,rain,"East Rutherford, NJ",MetLife Stadium,2010.0,outdoor,cold,"82,500",FieldTurf,40.813528,-74.074361
2,2010-09-19,2010,2,False,New York Jets,28.0,14.0,New England Patriots,NE,-3.0,39.5,MetLife Stadium,False,82.0,6.0,36.0,NaN,"East Rutherford, NJ",MetLife Stadium,2010.0,outdoor,cold,"82,500",FieldTurf,40.813528,-74.074361
3,2010-09-26,2010,3,False,New York Giants,10.0,29.0,Tennessee Titans,NYG,-3.0,43.5,MetLife Stadium,False,70.0,3.0,48.0,NaN,"East Rutherford, NJ",MetLife Stadium,2010.0,outdoor,cold,"82,500",FieldTurf,40.813528,-74.074361
4,2010-10-03,2010,4,False,New York Giants,17.0,3.0,Chicago Bears,NYG,-3.5,44,MetLife Stadium,False,59.0,1.0,52.0,NaN,"East Rutherford, NJ",MetLife Stadium,2010.0,outdoor,cold,"82,500",FieldTurf,40.813528,-74.074361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6865,2023-11-12,2023,10,False,Las Vegas Raiders,16.0,12.0,New York Jets,NYJ,-1.0,35.5,Allegiant Stadium,False,72.0,0.0,NaN,indoor,"Paradise, NV",Allegiant Stadium,2020.0,indoor,indoor,65000,Grass,36.090750,-115.183722
6866,2023-11-26,2023,12,False,Las Vegas Raiders,17.0,31.0,Kansas City Chiefs,KC,-9.0,42.5,Allegiant Stadium,False,72.0,0.0,NaN,indoor,"Paradise, NV",Allegiant Stadium,2020.0,indoor,indoor,65000,Grass,36.090750,-115.183722
6867,2023-12-10,2023,14,False,Las Vegas Raiders,0.0,3.0,Minnesota Vikings,MIN,-3.0,40.5,Allegiant Stadium,False,72.0,0.0,NaN,indoor,"Paradise, NV",Allegiant Stadium,2020.0,indoor,indoor,65000,Grass,36.090750,-115.183722
6868,2023-12-14,2023,15,False,Las Vegas Raiders,63.0,21.0,Los Angeles Chargers,LVR,-3.0,35.5,Allegiant Stadium,False,72.0,0.0,NaN,indoor,"Paradise, NV",Allegiant Stadium,2020.0,indoor,indoor,65000,Grass,36.090750,-115.183722
